# StyleGAN3 and Lucid Sonic Dreams 
Audioreactive videos with StyleGAN3

Made by [Artem Konevskikh](https://aiculedssul.net/)

Based on the 
[StyleGAN3](https://github.com/NVlabs/stylegan3),
[Lucid Sonic Dreams](https://github.com/mikaelalafriz/lucid-sonic-dreams),
[Lucid Sonic Dreams - Pytorch](https://github.com/NotNANtoN/lucid-sonic-dreams),
[Lucid Sonic Dreams by dvschultz](https://github.com/dvschultz/lucid-sonic-dreams)
and the notebook by [тоже моушн](https://t.me/too_motion)

In [ ]:
#@title GPU Check
#@markdown You can check which GPU you got. V100 is perfect, P100 and T4 is good, but if you got K80 its better to restart the notebook by selecting **Disconnect and delete runtime** in **Runtime** menu, because it is very slow
!nvidia-smi -L


In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive if you want to save the results there.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Lucid Sonid Dreams and StyleGAN3
from IPython.display import clear_output
%cd /content/
!git clone https://github.com/dvschultz/lucid-sonic-dreams
!pip install pygit2 mega.py ninja
%cd /content/lucid-sonic-dreams
!git clone https://github.com/NVlabs/stylegan3.git
!mv /content/lucid-sonic-dreams/stylegan3 /content/lucid-sonic-dreams/stylegan2
clear_output()
from lucidsonicdreams import LucidSonicDream

In [ ]:
#@title Download pretrained models
#@markdown Downloaded modeel will be saved in `/content/lucid-sonic-dreams` folder
from IPython.display import clear_output

model_to_download = "snowflakes" #@param ["snowflakes", "wikiart-t", "animals-t", "animals-r", "faces-t", "faces-r","artistic_faces-t","artistic_faces-r","cats","dogs","wild","minerals","acid_fluid","earth_view"]
if model_to_download == "wikiart-t":
  !gdown --id 18MOpwTMJsl_Z17q-wQVnaRLCUFZYSNkj # wikiart
elif model_to_download == "snowflakes":
  !gdown --id 12VK87A3AXsVksdwcsJBUOETyUyCdP0nX # snowflakes
elif model_to_download == "animals-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-afhqv2-512x512.pkl # animals-t
elif model_to_download == "animals-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl # animals-r
elif model_to_download == "faces-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl # faces-t
elif model_to_download == "faces-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl # faces-r
elif model_to_download == "artistic_faces-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-metfaces-1024x1024.pkl # artistic faces-t
elif model_to_download == "artistic_faces-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-metfaces-1024x1024.pkl # artistic faces-r
#elif model_to_download == "cosplay_faces":
#  !wget https://l4rz.net/cosplayface-snapshot-004000-18160-FID367.pkl # cosplay_faces
elif model_to_download == "cats":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqcat-512x512.pkl # cats
elif model_to_download == "dogs":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqdog-512x512.pkl # dogs
elif model_to_download == "wild":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqwild-512x512.pkl # wild
elif model_to_download == "minerals":
  !gdown --id 1YKkozuo2Ik_SGMwh-mFH0I9ZzCN2_gTn # minerals
elif model_to_download == "acid_fluid":
  !gdown --id 10IPazsui4eBPAiLi1p8iMSqL9h0hcqH0 # acid fluid
elif model_to_download == "earth_view":
  !gdown --id 15ypXiZatt2zYvA805vKoMbjH-iU-2IFD # earth view

clear_output()

In [ ]:
#@title Generate music video
#@markdown Music
music = '/content/music.mp3' #@param {type:"string"}

#@markdown Pretrained model. Either yours or the one you downloaded in previous cell
model = '/content/lucid-sonic-dreams/snowflakes1024.pkl' #@param {type:"string"}
#@markdown Output filename. Don't forget to rename next time you run this cell
final_video_name = '/content/LSD_animation_test.mp4' #@param {type:"string"}

#@markdown Starting timestamp in seconds.
start = 0 #@param {type:"integer"}
#@markdown Video duration in seconds. If none is passed, full duration of audio will be used.
duration = 0 #@param {type:"integer"}
duration = None if duration==0 else duration
#@markdown Self-explanatory. Low resolutions are recommended for "trial" renders. If 0 is passed, unaltered high-resolution images will be used.
resolution = 800 #@param {type:"integer"}
resolution = None if resolution==0 else resolution
#@markdown FPM stands for "Frames per Minute". This determines how many images are initialized - the more there are, the faster the visuals morph. If `speed_fpm = 0`, then only one image is initialized, and that single image reacts to the audio. In this case, there will be no motion during silent parts of the audio.
speed_fpm = 12 #@param {type:"integer"}
#@markdown Video Frames Per Second.
fps = 30 #@param {type:"integer"}
#@markdown The "strength" of the motion. It is recommended to keep this between 0 and 2.
motion_react = 0.8 #@param {type: "slider", min: 0.0, max: 2.0, step: 0.05}
#@markdown Degree of randomness of motion. Higher values will typically prevent the video from cycling through the same visuals repeatedly. Must range from 0 to 1.
motion_randomness = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.05}
#@markdown The "strength" of the pulse. It is recommended to keep this between 0 and 2.
pulse_react = 1 #@param {type: "slider", min: 0.0, max: 2.0, step: 0.05}
#@markdown Random seed
random_seed=1651 #@param {type:"slider", min:0, max:9999, step:1}

#@markdown You can read detailed description of the other parameters in the [original colab](https://colab.research.google.com/drive/1Y5i50xSFIuN3V4Md8TB30_GOAtts7RQD#scrollTo=GuLiMg9CUXvG)

L = LucidSonicDream(song = music, #mp3 here
                    style = model) #pkl path here

L.hallucinate(file_name = final_video_name,
              start = start,
              duration = duration,
              resolution = resolution,  
              speed_fpm = speed_fpm,
              fps = fps,
              motion_react = motion_react,
              motion_randomness = motion_randomness,
              pulse_react = pulse_react,
              random_seed = random_seed,
              ) 